# **4. cvičení - Substituční matice (4.3.2021)**

- slouží jako model substitucí - některé NT/AK se mění v jiné často, jiné substituce jsou vzácné → různé změny mají různou pravděpodobnost
- vyjadřuje míru substitucí mezi nukleotidy nebo aminokyselinami
- substituce v DNA
  - tranzice x transverze
  - synonymní x nesynonymní

- substituce AMK
  - nejčastěji substituce za strukturálně podobnou AMK
  
- slouží k ohodnocení zarovnání, což je nalezení podobností mezi sekvencemi


## Substituční matice pro NT - velikost 4x4

### Matice identity
- nukleotidy nejsou schopné zmutovat do jiného nukleotidu





<img src="https://drive.google.com/uc?export=view&id=1oQad3N-MO5jbZ5a9Dy7MJYm_I2YAsbEz">



### NUC44
- shoda nukleotidů = 5
- substituce = -4



<img src="https://drive.google.com/uc?export=view&id=13lpmfaVc4cI_nVZhwhNwQQ57uBI-EiM7">

## Substituční matice pro AMK - velikost 20x20

### PAM (Point Accepted Mutation)
- PAM1 - za daný čas dojde v sekvenci ke změně 1 % AK 
- 1 pozice může zmutovat vícekrát
- hodnota v matici - pravděpodobnostní míra substituce AK na jinou AK
  - čím vyšší hodnota, tím vyšší míra substituce
  - kde jsou nejvyšší hodnoty?
- PAM s vysokým číslem pro méně podobné sekvence (evolučně vzdálené) a naopak

### BLOSUM (BLOck SUbstitution Matrix)
- číslo matice udává podobnost bloků zarovnaných sekvencí
- BLOSUM s vysokým číslem pro blízké sekvence (evolučně příbuzné) a naopak
- sestavení matice: <br>
  1) popis a sestavení bloku - W, S, N <br>
  2) příprava komponent pro sestavení matice - f<sub>i,j</sub>, q<sub>i,j</sub>, p<sub>i</sub><br>
  3) vlastní konstrukce matice - b<sub>i,j</sub>, B<sub>i,j</sub>


**Úkol č. 1:** <br>
a) Popište, co je to substituce.<br>
b) Na jaké typy se substituce DNA dělí? Každý typ vysvětlete.

**Odpověď:**

a) Substituce je změna nukleotidu/aminokyseliny na jiný nukleotid/aminokyselinu.

b) Tranzice - změna nukleotidu purinový-purinový nebo pyrimidinový-pyrimidinový. Transverze - změna nukleotidu purinový-pyrimidinový. Synonymní - vytvoří kodon, který kóduje stejnou aminokyselinu jako původní kodon. Nesynonymní - vytvoří kodon, který kóduje jinou aminokyselinu než původní kodon.

**Úkol č. 2:** <br>
a) Popište, k čemu slouží matice PAM.<br>
b) Vysvětlete zkratku PAM1.

**Odpověď:**

a) První používaná substituční matice pro porovnávání proteinů.

b) Za daný čas dojde v sekvenci ke změně 1 % AK.

**Úkol č. 3:** <br>
Vytvořte funkci, který spočítá matici BLOSUM pro zadaný set sekvencí.<br>
Vstup: zarovnané sekvence<br>
Výstup: matice BLOSUM

In [ ]:
pip install biopython

In [ ]:
import numpy
import math

def blosum(blok):
  S = len(blok)
  W = len(blok[0])
  N = (S*(S-1)/2)*W

  s = ''.join(blok)
  znaky = sorted(set(s))

  D = len(znaky) 
  blok_list = []
  f = numpy.zeros(shape=(D, D), dtype=float)

  for j in range(S):
    pom = (list(blok[j]))
    blok_list.append(pom)
  
  rozd_pary = []

  for i in reversed(range(1, D+1)):
    zero = []
    for k in range(i):
      zero += [0]
    rozd_pary.append(zero)  

  for pos in range(len(blok_list[0])):
    for str1 in range(len(blok_list)):
      for str2 in range(str1+1, len(blok_list)):
        ch1 = blok_list[str1][pos]
        ch2 = blok_list[str2][pos]
        if ch1 == ch2:
          index_min = znaky.index(ch1)
          rozd_pary[index_min][0] += 1
        elif ch1 < ch2:
          index_min = znaky.index(ch1)
          index_max = znaky.index(ch2)
          rozd_pary[index_min][index_max - index_min] += 1

        elif ch1 > ch2:
          index_min = znaky.index(ch2)
          index_max = znaky.index(ch1)
          rozd_pary[index_min][index_max - index_min] +=1

  i = 0
  for st in range(len(rozd_pary)):
    for col in range(len(rozd_pary[st])):
      f[st][col+i] += rozd_pary[st][col] 
    i += 1

  for st in range(D):
    for col in range(D):
      if st > col:
        f[st][col] = f[col][st]
  print('Četnosti znaků jsou \n', f)

  norm_f = numpy.zeros((D, D), dtype=float)
  for st in range(D):
    for col in range(D):
      norm_f[st][col] = f[st][col] / N
  print('Normovaná matice četnosti znaků je \n', norm_f)

  marg_f = numpy.zeros(D, dtype=float)
   
  for i in range(D):
    marg_f[i] = sum(norm_f[i])/2 + norm_f[i][i]/2

  print('Marginální součty jsou: \n', marg_f)

  b = numpy.zeros((D, D), dtype=float)
  for st in range(D):
    for col in range(D):
      if st != col:
        b[st][col] += norm_f[st][col] / (2*marg_f[st]*marg_f[col])
      else:
        b[st][col] += norm_f[col][col] / marg_f[col]**2

  blos = numpy.zeros((D, D), dtype=float)
  for st in range(D):
    for col in range(D):
      blos[st][col] = round(3 * math.log2(b[st][col]))
  print('BLOSUM matice je: \n',blos)

vstup = ['CAABAB', 'BBABCB', 'AAABCB']
blosum(vstup)

Četnosti znaků jsou 
 [[4. 3. 3.]
 [3. 6. 1.]
 [3. 1. 1.]]
Normovaná matice četnosti znaků je 
 [[0.22222222 0.16666667 0.16666667]
 [0.16666667 0.33333333 0.05555556]
 [0.16666667 0.05555556 0.05555556]]
Marginální součty jsou: 
 [0.38888889 0.44444444 0.16666667]
BLOSUM matice je: 
 [[ 2. -3.  1.]
 [-3.  2. -4.]
 [ 1. -4.  3.]]


VSTUP: ['CAABAB', 'BBABCB', 'AAABCB']

VÝSTUP: </br>
Četnosti znaků jsou  [[4. 3. 3.]</br>
[3. 6. 1.]</br>
 [3. 1. 1.]]</br>
Normovaná matice četnosti znaků je  [[0.22222222 0.16666667 0.16666667]</br>
 [0.16666667 0.33333333 0.05555556]</br>
 [0.16666667 0.05555556 0.05555556]]</br>
Marginální součty jsou:  [0.38888889 0.44444444 0.16666667]</br>
BLOSUM matice je:  [[ 2. -3.  1.]</br>
 [-3.  2. -4.]</br>
 [ 1. -4.  3.]]</br>